In [1]:
%reset -fs

In [2]:
import pandas as pd
import numpy as np
import time, os
from bs4 import BeautifulSoup
import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [3]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver

In [4]:
driver = webdriver.Chrome(chromedriver)

/var/folders/36/w8jclnw52p3d1qr2544j3lth0000gn/T/ipykernel_6521/2060176432.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [5]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/'
query = '?date=' + datetime.date(2021, 10, 19).strftime('%Y-%m-%d')
bet_query = bet_search + query

In [6]:
base = datetime.datetime.today()
delta = base - datetime.datetime(2021, 10, 19)
date_list = [base - datetime.timedelta(days=x) for x in range(delta.days+1)]
date_list = [d.strftime('%Y-%m-%d') for d in date_list]

In [7]:
bet_query = bet_search + '?date=' + '2021-12-20'
driver.get(bet_query)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
#run this initially; then don't run again if you just want to keep adding to it
final_df = pd.DataFrame()

In [9]:
# Tip of the hat to Rita for helping with these two functions !
def get_site_rb(date):
    bet_query = bet_search + '?date=' + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [10]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = final_df.append(pd.DataFrame(data))
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df


In [11]:
#run this and then close the cookies pop-up!
for i in date_list:
    
    if i == '2021-12-24':
        continue
    if i == '2021-11-25':
        continue
    if i == '2022-02-18':
        continue
    if i == '2022-02-19':
        continue
    if i == '2022-02-20':
        continue
    if i == '2022-02-21':
        continue
    if i == '2022-02-22':
        continue
    if i == '2022-02-23':
        continue
    get_site_rb(i)

/var/folders/36/w8jclnw52p3d1qr2544j3lth0000gn/T/ipykernel_6521/2734737161.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
/var/folders/36/w8jclnw52p3d1qr2544j3lth0000gn/T/ipykernel_6521/1303484502.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(pd.DataFrame(data))
/var/folders/36/w8jclnw52p3d1qr2544j3lth0000gn/T/ipykernel_6521/1303484502.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
/var/folders/36/w8jclnw52p3d1qr2544j3lth0000gn/T/ipykernel_6521/1303484502.py:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_eleme

In [12]:
final_df

,0,1,2,3,4,5,6,7,8,9
0,1 T. Theo Maledon,OKC - G,vs. ATL,Points,18.50 (-120)11.80Diff: -6.70 Line: 18.5...,18.5,-120 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
1,2 D. Desmond Bane,MEM - G,@ SAS,Points,21.50 (-108)16.60Diff: -4.90 Line: 21.5...,21.5,-108 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
2,3 D. Davion Mitchell,SAC - G,@ HOU,Points,22.50 (-117)17.70Diff: -4.80 Line: 22.5...,22.5,-117 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
3,4 . Bogdan Bogdanovic,ATL - G,@ OKC,Points,18.50 (-125)14.30Diff: -4.20 Line: 18.5...,18.5,-125 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
4,5 G. Garrison Mathews,HOU - G,vs. SAC,Points,9.50 (-105)13.50Diff: 4.00 Line: 9.50 C...,9.5,-105 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
...,...,...,...,...,...,...,...,...,...,...
21,22 G. Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1141-1099 (5...
22,23 N. Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1141-1099 (5...
23,24 D. DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1141-1099 (5...
24,25 P. Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1141-1099 (5...


In [13]:
driver.close()

In [14]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [15]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [16]:
final_df.reset_index(inplace=True, drop=True)

In [18]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy
0,Theo Maledon,OKC - G,vs. ATL,Points,18.50 (-120)11.80Diff: -6.70 Line: 18.5...,18.5,-120 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
1,Desmond Bane,MEM - G,@ SAS,Points,21.50 (-108)16.60Diff: -4.90 Line: 21.5...,21.5,-108 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
2,Davion Mitchell,SAC - G,@ HOU,Points,22.50 (-117)17.70Diff: -4.80 Line: 22.5...,22.5,-117 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
3,Bogdan,ATL - G,@ OKC,Points,18.50 (-125)14.30Diff: -4.20 Line: 18.5...,18.5,-125 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
4,Garrison Mathews,HOU - G,vs. SAC,Points,9.50 (-105)13.50Diff: 4.00 Line: 9.50 C...,9.5,-105 TBD,TBD,03-30-2022,Last 7 days Top 15 Accuracy: 47-55 (46%) Sea...
...,...,...,...,...,...,...,...,...,...,...
14409,Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1141-1099 (5...
14410,Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1141-1099 (5...
14411,DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1141-1099 (5...
14412,Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1141-1099 (5...


In [19]:
team = final_df.Team.str.split(" ", expand = True)

In [20]:
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [21]:
opp = final_df.Opp.str.split(" ", expand = True)

In [22]:
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [23]:
bet_info = final_df.Line.str.split(" ", expand = True)

In [24]:
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')

In [25]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [27]:
final_df = final_df.copy()

In [28]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))

In [29]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [30]:
final_df['seven_day_accuracy'] = accuracies[8]

In [31]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [32]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [33]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [34]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [35]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [36]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [37]:
final_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2022-03-30,Theo Maledon,OKC,G,ATL,vs.,18.50,-120,11.80,-6.70,18.5,TBD,u,0.46
1,2022-03-30,Desmond Bane,MEM,G,SAS,@,21.50,-108,16.60,-4.90,21.5,TBD,u,0.46
2,2022-03-30,Davion Mitchell,SAC,G,HOU,@,22.50,-117,17.70,-4.80,22.5,TBD,u,0.46
3,2022-03-30,Bogdan,ATL,G,OKC,@,18.50,-125,14.30,-4.20,18.5,TBD,u,0.46
4,2022-03-30,Garrison Mathews,HOU,G,SAC,vs.,9.50,-105,13.50,4.00,9.5,TBD,o,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14409,2021-10-19,Grayson Allen,MIL,G,BKN,vs.,10.50,-113,11.23,0.73,10.5,10 (under),o,0.50
14410,2021-10-19,Nemanja Bjelica,GSW,F,LAL,@,6.50,-120,6.87,0.37,6.5,15 (over),o,0.50
14411,2021-10-19,DeAndre Jordan,LAL,C,GSW,vs.,4.50,-130,4.82,0.32,4.5,2 (under),o,0.50
14412,2021-10-19,Pat Connaughton,MIL,G,BKN,vs.,7.50,-111,7.19,-0.31,7.5,20 (over),u,0.50


In [38]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [39]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [40]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [41]:
sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11 (under),o,0.47
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5 (under),o,0.45
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14 (over),u,0.45
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12 (under),o,0.45
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14 (over),o,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14409,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16 (over),o,0.46
14410,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13 (over),u,0.50
14411,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11 (over),u,0.46
14412,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0 (under),u,0.46


In [42]:
sorted_df = sorted_df.replace('TBD', np.nan)

In [43]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [44]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x));sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11 (under),o,0.47
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5 (under),o,0.45
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14 (over),u,0.45
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12 (under),o,0.45
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14 (over),o,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14409,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16 (over),o,0.46
14410,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13 (over),u,0.50
14411,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11 (over),u,0.46
14412,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0 (under),u,0.46


In [45]:
results = sorted_df.Result.str.split(' ', expand=True);

In [46]:
sorted_df.Result = results[5]

In [47]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)

In [48]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u'))
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0']

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)

In [49]:
sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,NaN,o,0.47,0
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,NaN,o,0.45,0
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,NaN,u,0.45,0
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,NaN,o,0.45,0
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,NaN,o,0.42,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14409,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,NaN,o,0.46,0
14410,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,NaN,u,0.50,0
14411,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,NaN,u,0.46,0
14412,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,NaN,u,0.46,0


In [ ]:
sorted_df.to_csv('betting_data.csv')